In [2]:
# Import library
import pandas as pd
import numpy as np
import os, random, warnings, gc, psutil, datetime
from tqdm import tqdm_notebook, tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error
from math import sqrt

import lightgbm as lgbm
from catboost import CatBoostRegressor

from glob import glob
from IPython.display import display

import seaborn as sns
import matplotlib.pyplot as plt

# Set options
pd.set_option('max_columns',500)
pd.set_option('max_rows',500)
pd.options.display.max_colwidth = 300

warnings.filterwarnings('ignore')

%matplotlib inline
sns.set_palette('bright')

In [3]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything()

In [4]:
train = pd.read_pickle('../Create_Features/preprocessed_train.pickle')
test = pd.read_pickle('../Create_Features/preprocessed_test.pickle')

In [5]:
n_trn = 415423
target_col = '18~20_ride'

In [6]:
train.head()

,18~20_ride,bus_route_id,in_out,latitude,longitude,station_code,dayofweek,weekend,ride_total,takeoff_total,ride_go_to_work,takeoff_go_to_work,dis_jejusi,dis_seoquipo,bus_route_id_station_code,bus_route_id_station_code_weekend,date_fq_enc,station_code_fq_enc,bus_route_id_fq_enc,bus_route_id_station_code_fq_enc,date_bus_route_id_fq_enc,date_station_code_fq_enc,date_bus_route_id_station_code_fq_enc,7~8_ride_date_mean,7~8_ride_date_bus_route_id_mean,8~9_ride_date_mean,8~9_ride_date_bus_route_id_mean,9~10_ride_date_mean,9~10_ride_date_bus_route_id_mean,station_sequence,station_reverse_sequence,weekday,is_national_holiday,getin_total,morning_getin,morning_takeoff,noon_getin,noon_takeoff,station_morning_getin_sum,station_morning_takeoff_sum,bus_route_getin_sum,bus_route_takeoff_sum,station_morning_getin_mean,station_morning_takeoff_mean,bus_route_getin_mean,bus_route_takeoff_mean,kmeans1,kmeans2,regular_commuter_count,afternoon_takeoff,next_bus_time_diff,getin_user_count1_morning,getin_user_count2_morning,takeoff_user_count1_noon,takeoff_user_count2_noon,hourly_rain,prev_daily_rain,hourly_cloud,latlong_second,total_population,man_population,woman_population,avg_time_diff,passengers_in,passengers_out,latitude_rank,longitude_rank
0,0.0,0,1,33.48990,126.49373,322,6,1,16.0,0.0,8.0,0.0,2.954920,26.256744,31053,53745,11538,46,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,1,25,6,0,16.0,3.0,0.0,13.0,0.0,3.0,0.0,50.0,8.0,3.000000,0.000000,2.0,0.32,15,9,9.0,0.0,3650.490741,7.0,1.0,NaN,NaN,0.2,0.0,88,2411,43217.0,21189.0,22028.0,3113.015748,15.239130,1.909091,404772.5,214964.5
1,5.0,0,1,33.48944,126.48508,335,6,1,22.0,0.0,10.0,0.0,3.720275,26.403025,31054,53747,11538,2303,1189,46,25,45,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,2,24,6,0,22.0,9.0,0.0,13.0,0.0,183.0,116.0,50.0,8.0,4.066667,2.577778,2.0,0.32,15,9,466.0,0.0,3650.490741,197.0,NaN,95.0,NaN,0.2,0.0,88,2403,43217.0,21189.0,22028.0,1837.940774,594.000000,428.000000,396531.5,189710.0
2,2.0,0,1,33.48181,126.47352,408,6,1,4.0,0.0,3.0,0.0,5.036124,25.893305,31057,53753,11538,1154,1189,46,25,21,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,3,23,6,0,4.0,2.0,0.0,2.0,0.0,71.0,11.0,50.0,8.0,3.380952,0.523810,2.0,0.32,15,45,164.0,0.0,3650.490741,76.0,NaN,11.0,2.0,0.2,0.0,88,2347,56223.0,27761.0,28462.0,2448.248012,166.913043,28.000000,342909.5,156930.5
3,53.0,0,0,33.50577,126.49252,1448,6,1,79.0,0.0,49.0,0.0,2.864166,27.997494,31020,53682,11538,49,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,4,22,6,0,79.0,23.0,0.0,56.0,0.0,23.0,0.0,50.0,8.0,23.000000,0.000000,2.0,0.32,4,3,2.0,0.0,3650.490741,49.0,NaN,NaN,NaN,0.2,0.0,88,2980,15673.0,7904.0,7769.0,3961.540412,89.130435,3.600000,518315.0,210660.0
4,0.0,0,0,33.25579,126.41260,1510,6,1,0.0,1.0,0.0,1.0,29.040353,13.574693,31022,53686,11538,386,1189,39,25,10,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,5,21,6,0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,50.0,8.0,0.100000,0.500000,2.0,0.32,85,136,3.0,0.0,3650.490741,4.0,NaN,5.0,NaN,0.2,0.0,88,562,4414.0,2252.0,2162.0,1058.012605,6.847826,27.456522,98750.5,93507.5


In [7]:
# Before modeling
train_set = train.drop(target_col,1)
test_set = test.drop(target_col,1)

train_label = train[target_col]
test_label = test[target_col]

In [8]:
n_rounds = 100000
n_splits = 5

cat_params = {
        'n_estimators': n_rounds,
        'learning_rate': 0.05,
        'eval_metric': 'RMSE',
        'loss_function': 'RMSE',
        'random_seed': 42,
        'metric_period': 500,
        'od_wait': 500,
        'task_type': 'GPU',
       'l2_leaf_reg' : 3,
        'depth': 8,
    }

In [9]:
train.head()

,18~20_ride,bus_route_id,in_out,latitude,longitude,station_code,dayofweek,weekend,ride_total,takeoff_total,ride_go_to_work,takeoff_go_to_work,dis_jejusi,dis_seoquipo,bus_route_id_station_code,bus_route_id_station_code_weekend,date_fq_enc,station_code_fq_enc,bus_route_id_fq_enc,bus_route_id_station_code_fq_enc,date_bus_route_id_fq_enc,date_station_code_fq_enc,date_bus_route_id_station_code_fq_enc,7~8_ride_date_mean,7~8_ride_date_bus_route_id_mean,8~9_ride_date_mean,8~9_ride_date_bus_route_id_mean,9~10_ride_date_mean,9~10_ride_date_bus_route_id_mean,station_sequence,station_reverse_sequence,weekday,is_national_holiday,getin_total,morning_getin,morning_takeoff,noon_getin,noon_takeoff,station_morning_getin_sum,station_morning_takeoff_sum,bus_route_getin_sum,bus_route_takeoff_sum,station_morning_getin_mean,station_morning_takeoff_mean,bus_route_getin_mean,bus_route_takeoff_mean,kmeans1,kmeans2,regular_commuter_count,afternoon_takeoff,next_bus_time_diff,getin_user_count1_morning,getin_user_count2_morning,takeoff_user_count1_noon,takeoff_user_count2_noon,hourly_rain,prev_daily_rain,hourly_cloud,latlong_second,total_population,man_population,woman_population,avg_time_diff,passengers_in,passengers_out,latitude_rank,longitude_rank
0,0.0,0,1,33.48990,126.49373,322,6,1,16.0,0.0,8.0,0.0,2.954920,26.256744,31053,53745,11538,46,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,1,25,6,0,16.0,3.0,0.0,13.0,0.0,3.0,0.0,50.0,8.0,3.000000,0.000000,2.0,0.32,15,9,9.0,0.0,3650.490741,7.0,1.0,NaN,NaN,0.2,0.0,88,2411,43217.0,21189.0,22028.0,3113.015748,15.239130,1.909091,404772.5,214964.5
1,5.0,0,1,33.48944,126.48508,335,6,1,22.0,0.0,10.0,0.0,3.720275,26.403025,31054,53747,11538,2303,1189,46,25,45,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,2,24,6,0,22.0,9.0,0.0,13.0,0.0,183.0,116.0,50.0,8.0,4.066667,2.577778,2.0,0.32,15,9,466.0,0.0,3650.490741,197.0,NaN,95.0,NaN,0.2,0.0,88,2403,43217.0,21189.0,22028.0,1837.940774,594.000000,428.000000,396531.5,189710.0
2,2.0,0,1,33.48181,126.47352,408,6,1,4.0,0.0,3.0,0.0,5.036124,25.893305,31057,53753,11538,1154,1189,46,25,21,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,3,23,6,0,4.0,2.0,0.0,2.0,0.0,71.0,11.0,50.0,8.0,3.380952,0.523810,2.0,0.32,15,45,164.0,0.0,3650.490741,76.0,NaN,11.0,2.0,0.2,0.0,88,2347,56223.0,27761.0,28462.0,2448.248012,166.913043,28.000000,342909.5,156930.5
3,53.0,0,0,33.50577,126.49252,1448,6,1,79.0,0.0,49.0,0.0,2.864166,27.997494,31020,53682,11538,49,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,4,22,6,0,79.0,23.0,0.0,56.0,0.0,23.0,0.0,50.0,8.0,23.000000,0.000000,2.0,0.32,4,3,2.0,0.0,3650.490741,49.0,NaN,NaN,NaN,0.2,0.0,88,2980,15673.0,7904.0,7769.0,3961.540412,89.130435,3.600000,518315.0,210660.0
4,0.0,0,0,33.25579,126.41260,1510,6,1,0.0,1.0,0.0,1.0,29.040353,13.574693,31022,53686,11538,386,1189,39,25,10,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,5,21,6,0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,50.0,8.0,0.100000,0.500000,2.0,0.32,85,136,3.0,0.0,3650.490741,4.0,NaN,5.0,NaN,0.2,0.0,88,562,4414.0,2252.0,2162.0,1058.012605,6.847826,27.456522,98750.5,93507.5


In [10]:
cat_cols = list(train_set)

In [11]:
for col in tqdm_notebook( list(train_set) ):
    train_set[col] = train_set[col].astype(np.str)
    test_set[col] = test_set[col].astype(np.str)


In [ ]:
split_col = 'bus_route_id'
len_seeds = 1

outer_oof_train = np.zeros( train.shape[0] )
outer_oof_test = np.zeros( test.shape[0] )

for _ in tqdm_notebook(range(len_seeds)):
    
    seed = random.randint(1, 100000)
    cat_params['random_seed'] = seed
    
    cv_list = []

    oof_train = np.zeros( train.shape[0] )
    final_test = np.zeros( test.shape[0] )

    kfolds = StratifiedKFold(n_splits = n_splits, shuffle=True, random_state=seed )
    
    for ind, (trn_ind, val_ind) in tqdm_notebook( enumerate(kfolds.split(train_set, train_set[split_col])) ):

        X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
        X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]
        
        model = CatBoostRegressor(**cat_params)
    
        X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
        X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]


        model.fit( X_train, y_train, eval_set = (X_valid, y_valid), 
                      cat_features  = cat_cols,
                      use_best_model=True,
                      verbose=True)

        valid_pred = model.predict(X_valid)
        test_pred  = model.predict(test_set)

        oof_train[val_ind] += valid_pred
        final_test += test_pred

        cv_list.append( sqrt(mean_squared_error(y_valid, valid_pred)) )

        print('='*80)

    final_test /= n_splits

    print(f"Average CV : {np.mean(cv_list)}")
    print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, oof_train))}")
    
    outer_oof_train += oof_train
    outer_oof_test += final_test
    
outer_oof_train /= len_seeds
outer_oof_test /= len_seeds

print(f"Overall for OOF: {sqrt(mean_squared_error(train_label, outer_oof_train))}")

In [ ]:
split_col = 'bus_route_id'
len_seeds = 5

outer_oof_train = np.zeros( train.shape[0] )
outer_oof_test = np.zeros( test.shape[0] )

for _ in tqdm_notebook(range(len_seeds)):
    
    seed = random.randint(1, 100000)
    cat_params['random_seed'] = seed
    
    cv_list = []

    oof_train = np.zeros( train.shape[0] )
    final_test = np.zeros( test.shape[0] )

    kfolds = StratifiedKFold(n_splits = n_splits, shuffle=True, random_state=seed )
    
    for ind, (trn_ind, val_ind) in tqdm_notebook( enumerate(kfolds.split(train_set, train_set[split_col])) ):

        X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
        X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]
        
        model = CatBoostRegressor(**cat_params)
    
        X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
        X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]


        model.fit( X_train, y_train, eval_set = (X_valid, y_valid), 
                      cat_features  = ['bus_route_id','station_code','weekday',\
                                                'kmeans1','kmeans2',
                                               'latlong_second',
                                              ] ,
                      use_best_model=True,
                      verbose=True)

        valid_pred = model.predict(X_valid)
        test_pred  = model.predict(test_set)

        oof_train[val_ind] += valid_pred
        final_test += test_pred

        cv_list.append( sqrt(mean_squared_error(y_valid, valid_pred)) )

        print('='*80)

    final_test /= n_splits

    print(f"Average CV : {np.mean(cv_list)}")
    print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, oof_train))}")
    
    outer_oof_train += oof_train
    outer_oof_test += final_test
    
outer_oof_train /= len_seeds
outer_oof_test /= len_seeds

print(f"Overall for OOF: {sqrt(mean_squared_error(train_label, outer_oof_train))}")

In [10]:
# PostProcessing
outer_oof_train = [x if x>0 else 0 for x in  outer_oof_train]
outer_oof_test = [x if x>0 else 0 for x in  outer_oof_test]

print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, outer_oof_train))}")

RMSE for OOF: 2.132076609656555


In [11]:
df_oof = pd.read_csv('../raw_dataset/train.csv', usecols = ['id','18~20_ride'])
df_oof['18~20_ride'] = outer_oof_train

df_oof.to_csv('../oof/cat_5_seeds_stractified5k_bus_route_id.csv',index=False)

In [12]:
df_sub = pd.read_csv('../raw_dataset/submission_sample.csv')
df_sub['18~20_ride'] = outer_oof_test

df_sub.to_csv('../submission/cat_5_seeds_stractified5k_bus_route_id.csv',index=False)